# AI Me - LoRA Fine-tuning on Colab

This notebook runs LoRA fine-tuning on Google Colab with GPU acceleration.

## Setup Instructions

1. **Set your Hugging Face API key** in the cell below
2. **Upload your dataset** (ai_me_chat.zip) when prompted
3. **Run all cells** to start training

## Requirements
- Colab with GPU runtime (T4, V100, or A100)
- Hugging Face account with access to Llama models
- Dataset prepared by `prepare_dataset.py`

## 1. Check GPU and Install Dependencies

In [ ]:
# Check GPU
!nvidia-smi

# Install stable versions
!pip -q install -U "transformers>=4.43.3" "accelerate>=0.32.0" "peft>=0.11.1" \
  "datasets>=2.20.0" "tokenizers>=0.19.1" "bitsandbytes>=0.43.2" sentencepiece huggingface_hub

## 2. Import Libraries and Setup

In [ ]:
from huggingface_hub import login, InferenceClient
from google.colab import userdata, files
from datasets import load_from_disk
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# Check PyTorch version
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")

## 3. Login to Hugging Face

**Important**: Set your HF key in Colab's userdata first:
1. Go to Colab menu: Runtime → Manage secrets
2. Add a new secret with key: `hf_key`
3. Value: Your Hugging Face API token
4. Click 'Add secret'

In [ ]:
# Login to Hugging Face
try:
    hf_key = userdata.get('hf_key')
    if hf_key:
        login(hf_key)
        print("✅ Successfully logged in to Hugging Face")
    else:
        print("❌ HF key not found in userdata. Please set it in Runtime → Manage secrets")
        print("Key name should be: hf_key")
except Exception as e:
    print(f"❌ Error logging in: {e}")
    print("Please check your HF key in userdata")

## 4. Clone Repository

In [ ]:
%cd /content
!rm -rf ai_me
!git clone https://github.com/sandronatchkebia/ai_me.git
%cd ai_me
print(f"Repository cloned to: {!pwd}")

## 5. Upload and Extract Dataset

**Upload your `ai_me_chat.zip` file when prompted below.**

This should contain the dataset prepared by `prepare_dataset.py` in the `fine_tuning/dataset/ai_me_chat/` directory.

In [ ]:
# Upload dataset
print("📁 Please upload your ai_me_chat.zip file:")
uploaded = files.upload()

# Extract dataset
print("📂 Extracting dataset...")
!unzip -o ai_me_chat.zip -d /content/ai_me/fine_tuning/dataset/ >/dev/null

# Verify dataset
print("🔍 Verifying dataset...")
ds = load_from_disk("/content/ai_me/fine_tuning/dataset/ai_me_chat")
print(f"Dataset loaded: {len(ds['train'])} training samples, {len(ds['validation'])} validation samples")

## 6. Start LoRA Training

The training will use:
- **Model**: Meta-Llama-3.1-8B-Instruct
- **Method**: LoRA with 4-bit quantization
- **Training**: 2 epochs, learning rate 1.5e-4
- **Sequence length**: 2048 tokens
- **Batch size**: 2 per device, 8 gradient accumulation steps

In [ ]:
# Start training
print("🚀 Starting LoRA training...")
print("This may take several hours depending on your GPU.")

!python fine_tuning/train_lora.py \
  --model_id "meta-llama/Meta-Llama-3.1-8B-Instruct" \
  --dataset_dir "/content/ai_me/fine_tuning/dataset/ai_me_chat" \
  --output_dir "/content/ai_me/fine_tuning/out/ai_me_lora_llama3p1_8b" \
  --epochs 2.0 --learning_rate 1.5e-4 --max_seq_len 2048 \
  --per_device_train_batch_size 2 --gradient_accumulation_steps 8 \
  --save_steps 400 --save_total_limit 2 --logging_steps 20 \
  --load_4bit --bf16

## 7. Download Results

After training completes, download your fine-tuned LoRA model.

In [ ]:
# Create zip file of results
print("📦 Creating results archive...")
!cd /content/ai_me/fine_tuning/out && zip -r ai_me_lora_latest.zip ai_me_lora_llama3p1_8b >/dev/null

# Download results
print("⬇️ Downloading results...")
files.download("/content/ai_me/fine_tuning/out/ai_me_lora_latest.zip")

print("✅ Training complete! Your LoRA model has been downloaded.")

## Optional: Save to Google Drive

If you want to save the results to Google Drive instead:

In [ ]:
# Mount Google Drive (uncomment if needed)
# from google.colab import drive
# drive.mount('/content/drive')

# Save to Drive (uncomment if needed)
# !zip -r /content/drive/MyDrive/ai_me_lora_latest.zip /content/ai_me/fine_tuning/out/ai_me_lora_llama3p1_8b
# print("💾 Results saved to Google Drive")